In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from joblib import dump

import os
import glob
import time
import h5py
import numpy as np
import pandas as pd
from joblib import load
import matplotlib.pyplot as plt
from collections import deque
from datetime import datetime

# initial data for training purpose

In [2]:
# Lists of the same length
bpm_intensities = [14182.8, 9569.7, 9500.9, 4515.1, 6700.5, 2311.5, 2333.0, 
                      5896.5, 21304.9, 22182.8, 33490.4, 26747.4, 27292.1, 
                      609.4, 9853.1, 4644.7, 58453.6, 53575.8, 11574.3, 7441.3, 
                      4690.2, 3773.1, 3627.0, 19376.0, 4818.2, 5972.3, 5509.7, 
                      6015.2, 8707.6, 15630.3, 18290.8, 14188.1, 13908.0, 
                      13151.9, 13757.1, 13632.2, 12964.6, 13924.3, 13520.7, 
                      13860.3, 14141.4, 13108.2, 13588.6, 14447.2, 12799.3, 
                      12399.3, 9724.8, 10447.9, 9493.1, 9830.0, 10709.5, 
                      10769.5, 11801.5, 9650.8, 10437.6, 10292.1, 10307.3, 
                      10696.8, 10029.3, 10725.9]   # list of floats (already averaged/summed)
scintillator_signals = [1e7]*60    # list of floats (from the .h5 extraction)
fc_charges = [115, 111, 20, 15, 110, 111, 118, 115, 109, 106,
                    115, 111, 20, 15, 110, 111, 118, 115, 109, 106,
                     115, 111, 20, 15, 110, 111, 118, 115, 109, 106,
                     115, 111, 20, 15, 110, 111, 118, 115, 109, 106,
                     115, 111, 20, 15, 110, 111, 118, 115, 109, 106,
                     115, 111, 20, 15, 110, 111, 118, 115, 109, 106]   # list of the FC charge values (when FC was ON)

X = np.column_stack((bpm_intensities, scintillator_signals))  # shape: (N, 2)
y = np.array(fc_charges)                                      # shape: (N,)

# to train the model

In [4]:
# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the LG model
#model = LinearRegression()
#model.fit(X_train, y_train)

# For RF modelling (better prediction result proven)
model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)

# Save the model
dump(model, "fc_charge_predictor.joblib")
print("Model saved as fc_charge_predictor.joblib")

Model saved as fc_charge_predictor.joblib


#  Live Prediction from h5 Files (this section can be omitted)

In [ ]:
# Load trained model
model = load("fc_charge_predictor.joblib")
print("Model loaded.")

# Now the reference data when FC was OFF (although another approach used later below)
# instead of using this array to test the model and predict, direct extraction from *.h5 files opted.

data_fc_off = pd.DataFrame({
    'time': [
        '15:49:15.184', '15:50:55.974', '15:52:36.764', '15:54:17.554', 
               '15:55:58.344', '15:57:39.132', '15:59:19.923', '16:01:00.711',
               '16:02:41.601', '16:04:22.391', '16:06:03.179', '16:07:43.969',
               '16:09:24.759', '16:11:05.549', '16:12:46.338', '16:14:01.954',
               '16:16:07.916', '16:17:48.806', '16:19:29.594', '16:21:10.384',
               '16:22:51.172', '16:24:31.962', '16:26:12.751', '16:27:53.539',
               '16:29:34.329', '16:31:15.119', '16:53:45.189', '16:54:10.361',
               '17:33:28.414', '17:34:18.758'
    ],
    'bpm_intensity': [
        7651.2, 8243.2, 11403.2, 11817.6, 11656.0, 12249.6, 
                      9454.4, 11193.6, 10270.4, 13294.4, 9936.0, 13534.4, 
                      7491.2, 11462.4, 11867.2, 9673.6, 12752.0, 11964.8, 
                      11776.0, 12518.4, 9440.0, 12291.2, 13640.0, 12547.2, 
                      16499.2, 8926.4, 9393.6, 11868.8, 13124.8, 9448.0
    ]
})

# Convert time column to datetime.time objects for easy comparison
data_fc_off["time_obj"] = pd.to_datetime(data_fc_off["time"], format="%H:%M:%S.%f").dt.time
bpm_data_dict = dict(zip(data_fc_off["time_obj"], data_fc_off["bpm_intensity"]))

# Set up plot
plt.ion()  # for interactive plotting
fig, ax = plt.subplots(figsize=(10, 5))
times = deque(maxlen=100)
charges = deque(maxlen=100)
line, = ax.plot([], [], label="Predicted FC Charge", color='tab:blue')
ax.set_xlabel("Time")
ax.set_ylabel("FC Charge (pC)")
ax.set_title("Live Estimated FC Charge (based on BPM + Scintillator)")
ax.legend()
plt.tight_layout()


# Extracting timestamp from HDF5 filename (assuming format like: data_YYYYMMDD_HHMMSS.h5)
def extract_timestamp_from_filename(fname):
    base = os.path.basename(fname)
    time_part = base.split("_")[-1].replace(".h5", "")
    return datetime.strptime(time_part, "%H%M%S")


# Monitor folder
watch_dir = "/eos/experiment/awake/event_data/2025/04/28/"  # to be Changed to desired data folder

# Live monitoring logic
processed_files = set()  # Tracking processed files

while True:
    try:
        files = sorted([f for f in os.listdir(watch_dir) if f.endswith(".h5")])
        new_files = [f for f in files if f not in processed_files]

        for f in new_files:
            full_path = os.path.join(watch_dir, f)
            try:
                timestamp = extract_timestamp_from_filename(f)
                time_obj = timestamp.time()

                # Lookup BPM value
                if time_obj in bpm_data_dict:
                    bpm = bpm_data_dict[time_obj]
                    
                    with h5py.File(full_path, 'r') as f:
                        elecimg2d = f['AwakeEventData/TT43.BTV.430042.DigiCam/ExtractionImage/image2D'][:]
                        scint = np.sum(elecimg2d)

                    #scint = extract_scintillator_signal(full_path)

                    if scint is not None:
                        predicted_charge = model.predict([[bpm, scint]])[0]
                        print(f"[{timestamp.strftime('%H:%M:%S.%f')[:-3]}] BPM: {bpm:.1f}, Scint: {scint:.1f} → FC: {predicted_charge:.2f} pC")

                        # Update live plot
                        times.append(timestamp)
                        charges.append(predicted_charge)
                        line.set_data(times, charges)
                        ax.relim()
                        ax.autoscale_view()
                        plt.draw()
                        plt.pause(0.1)

                else:
                    print(f"No BPM data found for time {time_obj}. Skipping...")

            except Exception as e:
                print(f"Error processing {f}: {e}")

            processed_files.add(f)

        time.sleep(1)

    except KeyboardInterrupt:
        print("Live monitoring stopped.")
        break


# updated code for directly reading the data from h5 files, for prediction

In [ ]:
# Code for looping over the files to read the bpm intensity values instead of reading them one by one

# Load trained model
model = load("fc_charge_predictor.joblib")

# Watch directory
watch_dir = "/eos/experiment/awake/event_data/2025/04/28/"

# Bookkeeping
processed_files = set()
timestamps = []
scintillator_signals = []
predicted_charges = []

# Plot setup
plt.ion()      # for interactive plotting
fig, ax = plt.subplots()

while True:
    #files = sorted(glob.glob(os.path.join(watch_dir, "*.h5")))
    files = sorted(glob.glob(os.path.join(watch_dir, "*.h5")))[:10]  # to lmit to first 10 files

    new_files = [f for f in files if f not in processed_files]

    for file_path in new_files:
        try:
            # Using the timestamp function
            timestamp = extract_timestamp_from_filename(file_path)
            time_obj = timestamp.time()
'''
            # Lookup BPM from dictionary (if parsing the values above in 'data_fc_off' array)
            if time_obj not in bpm_data_dict:
                print(f"No BPM match for {f}, skipping...")
                continue
            bpm = bpm_data_dict[time_obj]
'''         
            # or else read the intensity directly from the h5 files as attempted later below...
    
            start = time.time()

            # Reading scintillator data from the images
            start = time.time()
            with h5py.File(file_path, "r") as f:
                # Extract scintillator image
                img = f['AwakeEventData/TT43.BTV.430042.DigiCam/ExtractionImage/image2D'][:]
                signal = np.sum(img)
                
                # Read/extracting the BPM intensity directly from the h5 files
                if 'AwakeEventData/TT43.BPM.430028/Acquisition/sigma' in f:
                    sigma = f['AwakeEventData/TT43.BPM.430028/Acquisition/sigma'][:]
                    bpm_intensity = float(np.mean(sigma))
                else:
                    print(f"BPM sigma not found in {file_path}, skipping...")
                    continue
            print(f"Time to read + process: {time.time() - start:.2f}s")


        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            continue

        # Parse timestamp from filename
        fname = os.path.basename(file_path)
        try:
            timestamp = datetime.strptime(fname.split("_")[-1].replace(".h5", ""), "%H%M%S")
        except Exception:
            timestamp = datetime.now()

        # Prediction...
        start = time.time()

        X = [[signal, bpm_intensity]]      # maintaing the 2D input array shape
        charge = model.predict(X)[0]
        
        # Debug print - checking hits, intensity and prediction
        print(f"{timestamp} — Signal: {signal:.2e}, BPM: {bpm_intensity:.2f}, Predicted Charge: {charge:.3f} nC")
        
        print(f"Prediction took: {time.time() - start:.4f}s")

        # Store
        timestamps.append(timestamp)
        scintillator_signals.append(signal)
        predicted_charges.append(charge)

        # Keep last 300 (optional attempt...)
        if len(timestamps) > 300:
            timestamps = timestamps[-300:]
            scintillator_signals = scintillator_signals[-300:]
            predicted_charges = predicted_charges[-300:]

        # Plotting....
        start = time.time()
        
        #ax.clear()
        #ax.plot(timestamps, predicted_charges, label="Predicted FC Charge [nC]", color="blue")
        #ax.set_xlabel("Time")
        #ax.set_ylabel("Charge [nC]")
        #ax.set_title("Live FC Prediction")
        #ax.legend()
        #fig.autofmt_xdate()
        #plt.pause(0.01)
        
        # Only updating the plot every N(20) files, to make it faster....
        if len(processed_files) % 20 == 0:  # Update every 20 files
            ax.clear()
            ax.plot(timestamps, predicted_charges, label="Predicted FC Charge [nC]")
            ax.set_xlabel("Time")
            ax.set_ylabel("Charge [nC]")
            ax.set_title("Live FC Prediction from Scintillator Image")
            ax.legend()
            fig.autofmt_xdate()
            plt.pause(0.01)

        
            print(f"Plot took: {time.time() - start:.2f}s")

        processed_files.add(file_path)

    time.sleep(2)  # Check every 2 seconds


In [ ]:
# loading the trained model
model = load("fc_charge_predictor.joblib")

# For the mapping from timestamps to BPM intensities
# more of a lookup table...
bpm_data_dict = {
    '14:40:30.521': 14182.8,
    '14:41:35.306': 9569.7,
    '14:42:40.090': 
    # to be filled with bpm_intensities and timestamps
}

def extract_scintillator_signal(h5_path):
    with h5py.File(h5_path, 'r') as f:
        img2d = f['AwakeEventData/TT43.BTV.430042.DigiCam/ExtractionImage/image2D'][:]
        return np.sum(img2d)

def extract_timestamp_from_filename(filename):
    nanos_str = os.path.basename(filename).split('_')[0] #extracts the first part before the underscore, if the filename has underscore!
    nanos = int(nanos_str) #converts the string to an integer — this is a UNIX timestamp in nanoseconds
    return datetime.utcfromtimestamp(nanos / 1e9) # converts nanoseconds → seconds and returns a datetime object.

# Setup live plot
plt.ion()
fig, ax = plt.subplots(figsize=(10, 5))
times = deque(maxlen=100)
charges = deque(maxlen=100)
line, = ax.plot([], [], label="Predicted FC Charge", color='tab:blue')
ax.set_xlabel("Time")
ax.set_ylabel("FC Charge (pC)")
ax.set_title("Live Estimated FC Charge")
ax.legend()
plt.tight_layout()

# Monitor a folder
watch_dir = "./data"  # change to any other folder
processed_files = set()

while True:
    files = sorted([f for f in os.listdir(watch_dir) if f.endswith(".h5")])
    new_files = [f for f in files if f not in processed_files]

    for f in new_files:
        full_path = os.path.join(watch_dir, f)
        try:
            timestamp = extract_timestamp_from_filename(f)
            time_str = timestamp.strftime("%H:%M:%S.%f")[:-3]

            if time_str in bpm_data_dict:
                bpm = bpm_data_dict[time_str]
                #scint = extract_scintillator_signal(full_path)
                
                with h5py.File(full_path, 'r') as f:
                    elecimg2d = f['AwakeEventData/TT43.BTV.430042.DigiCam/ExtractionImage/image2D'][:]
                    scint = np.sum(elecimg2d)

                
                predicted_charge = model.predict([[bpm, scint]])[0]

                times.append(timestamp)
                charges.append(predicted_charge)

                # Update plot
                line.set_data(times, charges)
                ax.relim()
                ax.autoscale_view()
                plt.draw()
                plt.pause(0.1)

                print(f"[{time_str}] BPM: {bpm:.1f}, Scint: {scint:.1f} → FC: {predicted_charge:.2f} pC")
            else:
                print(f"Timestamp {time_str} not found in BPM mapping.")
        except Exception as e:
            print(f"Failed to process {f}: {e}")
        processed_files.add(f)

    time.sleep(1)